In [48]:
import requests 
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

In [29]:
base_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{base_url}?raw=1'
df = pd.read_csv(url)

In [31]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


# Q1

In [32]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

/usr/local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [33]:
answer_llm = df.iloc[0].answer_llm

In [36]:
embed0 = embedding_model.encode(answer_llm)

In [39]:
embed0[0]

-0.42244643

# Q2. Computing the dot product

In [43]:
evaluations_dot = []

In [44]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    answer_llm_embed = embedding_model.encode(row.answer_llm)
    answer_orig_embed = embedding_model.encode(row.answer_orig)
    evaluations_dot.append(answer_llm_embed @ answer_orig_embed)

100%|██████████| 1830/1830 [07:42<00:00,  3.95it/s]


In [49]:
np.percentile(evaluations_dot, 75)

32.38980484008789

# Q3. Computing the cosine

In [54]:
def cosine(e1, e2):
    norm_e1 = np.sqrt((e1 * e1).sum())
    e1_normed = e1 / norm_e1
    
    norm_e2 = np.sqrt((e2 * e2).sum())
    e2_normed = e2 / norm_e2
    
    return e1_normed @ e2_normed

In [55]:
evaluation_cosine = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    answer_llm_embed = embedding_model.encode(row.answer_llm)
    answer_orig_embed = embedding_model.encode(row.answer_orig)
    evaluation_cosine.append(cosine(answer_llm_embed, answer_orig_embed))

100%|██████████| 1830/1830 [03:17<00:00,  9.27it/s]


In [56]:
np.percentile(evaluation_cosine, 75)

0.8531118482351303

# Q4. Rouge

In [57]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df[df["document"] == "5170565b"]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
print(scores)

{'rouge-1': {'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}, 'rouge-2': {'r': 0.21621621621621623, 'p': 0.21621621621621623, 'f': 0.21621621121621637}, 'rouge-l': {'r': 0.3939393939393939, 'p': 0.3939393939393939, 'f': 0.393939388939394}}


# Q5. Average rouge score

In [59]:
np.mean([v["f"] for v in scores.values()])

0.35490034990035496

# Q6. Average rouge score for all the data points

In [64]:
f1_scores_rouge_2 = []
rouge_scorer = Rouge()

for i, row in tqdm(df.iterrows(), total=len(df)):
    f1_scores_rouge_2.append(rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]["rouge-2"]["f"])

100%|██████████| 1830/1830 [00:08<00:00, 211.18it/s]


In [65]:
np.mean(f1_scores_rouge_2)

0.17671704698262222